In [137]:
import pandas as pd
import numpy as np
import datetime
from datetime import date

In [140]:
#function to date race date, number of weeks to train, and generate 
def get_calendar(race_year, race_month, race_day, weeks):
    
    race_date = date(race_year, race_month, race_day+1)
    
    cal = weeks*7
    
    date_list = [race_date - datetime.timedelta(days=x) for x in range(1,(cal+1))]
    
    date_list.reverse()
    
    days = []

    for x in date_list:
        days.append(x.weekday())
        
    data = {'date': date_list, 'day_code': days}
    
    df_training_cal = pd.DataFrame(data)
    
    #create validation table for weekday codes/desc to join to training calendar
    weekdays = {'day_code': range(0,7), 'day_desc': ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']}
    df_weekdays = pd.DataFrame(weekdays)
    df_weekdays
    
    df_training_cal = pd.merge(df_training_cal, df_weekdays, how='left', on='day_code')
    
    #find first monday and crop calendar down to start on first monday
    first_mon = df_training_cal[df_training_cal.day_code == 0].index[0]
    df_training_cal = df_training_cal.iloc[first_mon:]
    
    #create column for week count
    week = []
    count = 0
    for index, row in df_training_cal.iterrows():
        if row.day_code == 0:
            count += 1
            week.append(count)
        else:
            week.append(count)
    df_training_cal['week']=week
    
    #create column for training phase
    block = df_training_cal.week.max() - 2
    base = np.ceil(block*0.4)
    speed = np.floor(block*0.6)

    phase= []

    for index, row in df_training_cal.iterrows():
        if row.week <= base:
            phase.append('base')
        elif row.week-base <= speed:
            phase.append('speed')
        else:
            phase.append('taper')
    df_training_cal['phase']=phase
    
    return df_training_cal

In [142]:
df = get_calendar(race_year=2023, race_month=11, race_day=19, weeks=16)
df

date  day_code day_desc  week  phase
0    2023-07-31         0      Mon     1   base
1    2023-08-01         1     Tues     1   base
2    2023-08-02         2      Wed     1   base
3    2023-08-03         3    Thurs     1   base
4    2023-08-04         4      Fri     1   base
..          ...       ...      ...   ...    ...
107  2023-11-15         2      Wed    16  taper
108  2023-11-16         3    Thurs    16  taper
109  2023-11-17         4      Fri    16  taper
110  2023-11-18         5      Sat    16  taper
111  2023-11-19         6      Sun    16  taper

[112 rows x 5 columns]